In [34]:
# importing necessary packages

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

import io
import time
import pprint
import pymongo
from pymongo import MongoClient
import pandas as pd

import psycopg2

In [35]:
# defining a universal pause time
SCROLL_PAUSE_TIME = 0.5

In [36]:
# creating a chrome drive instance
driver = webdriver.Chrome()

# getting the url
url = "https://en.wikipedia.org/wiki/List_of_London_Underground_stations"
driver.get(url)

# finding the element and scrolling to the element
table_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[1]')
driver.execute_script("arguments[0].scrollIntoView({'block':'start', 'inline':'nearest'})", table_element)


In [37]:
# creating a scraper
soup = BeautifulSoup(driver.page_source, "lxml")

# finding all the elements
names_table = soup.find_all('table', class_='wikitable sortable plainrowheaders col6center col7center col8center col10right jquery-tablesorter')

# creating a dataframe
df = pd.read_html(str(names_table))
df = pd.DataFrame(df[0])

df.head()

C:\Users\snkri\AppData\Local\Temp\ipykernel_34924\499981820.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(names_table))


,Station,Photograph,Line(s)[a],Local authority,Zone[b],Opened[5],Resited,Main line opened,Other name(s)[c][6][d],"Annual usage (millions, 2022)[7]",Area Served
0,Acton Town,NaN,DistrictPiccadilly,Ealing,3,1 Jul 1879,NaN,NaN,Mill Hill Park: 1879–1910,4.93,Acton
1,Aldgate,NaN,Met.[e]Circle,City of London,1,18 Nov 1876,NaN,NaN,NaN,6.90,Portsoken
2,Aldgate East,NaN,H&C[f]District,Tower Hamlets,1,6 Oct 1884,31 Oct 1938,NaN,Commercial Road: proposed before opening,10.23,Whitechapel
3,Alperton,NaN,Piccadilly[g],Brent,4,28 Jun 1903,NaN,NaN,Perivale-Alperton: 1903–1910,2.30,Alperton
4,Amersham,NaN,Met.,Bucking­ham­shire,9,1 Sep 1892,NaN,NaN,Amersham: 1892–1922Amersham & Chesham Bois: 19...,1.66,Amersham


In [38]:
# finding the google maps URLs to all the tube stations


# link to the website that generates the URLs 
linkgen_url = 'https://media.star-telegram.com/static/labs/GoogleMapLink2/index.html'

# Getting the URL
driver.get(linkgen_url)

# finding the searchbox
search_element = driver.find_element(By.XPATH, '//*[@id="autocomplete"]')

# function to get the URLs of the stations from the website
def link_generator(name):
        
        textToEnter = name + ' Underground station'
        
        driver.execute_script("arguments[0].value=\"" + textToEnter + "\"", search_element)

        submit = driver.find_element(By.XPATH, '/html/body/button')
        submit.click()

        soup = BeautifulSoup(driver.page_source, "html.parser")
        search_soup = soup.find(id="linkcopy")

        return search_soup.text


In [39]:
df.head()

,Station,Photograph,Line(s)[a],Local authority,Zone[b],Opened[5],Resited,Main line opened,Other name(s)[c][6][d],"Annual usage (millions, 2022)[7]",Area Served
0,Acton Town,NaN,DistrictPiccadilly,Ealing,3,1 Jul 1879,NaN,NaN,Mill Hill Park: 1879–1910,4.93,Acton
1,Aldgate,NaN,Met.[e]Circle,City of London,1,18 Nov 1876,NaN,NaN,NaN,6.90,Portsoken
2,Aldgate East,NaN,H&C[f]District,Tower Hamlets,1,6 Oct 1884,31 Oct 1938,NaN,Commercial Road: proposed before opening,10.23,Whitechapel
3,Alperton,NaN,Piccadilly[g],Brent,4,28 Jun 1903,NaN,NaN,Perivale-Alperton: 1903–1910,2.30,Alperton
4,Amersham,NaN,Met.,Bucking­ham­shire,9,1 Sep 1892,NaN,NaN,Amersham: 1892–1922Amersham & Chesham Bois: 19...,1.66,Amersham


In [40]:
# Getting the URLs 
links = []

for name in df['Station']:
        links.append(link_generator(name))

# creating new column with URLs
df['URL'] = links


# Converting the dataframe to a CSV file
df.to_csv('C:/Users/snkri/OneDrive/Desktop/tfl_project/tube_stations_list.csv')


In [41]:
df = pd.read_csv("C:/Users/snkri/OneDrive/Desktop/tfl_project/tube_stations_list.csv", index_col=[0])

In [42]:
df.head()

,Station,Photograph,Line(s)[a],Local authority,Zone[b],Opened[5],Resited,Main line opened,Other name(s)[c][6][d],"Annual usage (millions, 2022)[7]",Area Served,URL
0,Acton Town,NaN,DistrictPiccadilly,Ealing,3,1 Jul 1879,NaN,NaN,Mill Hill Park: 1879–1910,4.93,Acton,https://www.google.com/maps?q=Acton+Town+Under...
1,Aldgate,NaN,Met.[e]Circle,City of London,1,18 Nov 1876,NaN,NaN,NaN,6.90,Portsoken,https://www.google.com/maps?q=Aldgate+Undergro...
2,Aldgate East,NaN,H&C[f]District,Tower Hamlets,1,6 Oct 1884,31 Oct 1938,NaN,Commercial Road: proposed before opening,10.23,Whitechapel,https://www.google.com/maps?q=Aldgate+East+Und...
3,Alperton,NaN,Piccadilly[g],Brent,4,28 Jun 1903,NaN,NaN,Perivale-Alperton: 1903–1910,2.30,Alperton,https://www.google.com/maps?q=Alperton+Undergr...
4,Amersham,NaN,Met.,Bucking­ham­shire,9,1 Sep 1892,NaN,NaN,Amersham: 1892–1922Amersham & Chesham Bois: 19...,1.66,Amersham,https://www.google.com/maps?q=Amersham+Undergr...
